In [2]:
import pandas as pd
import tensorflow as tf 

train_data = pd.read_csv('/code/mnist/train.csv')
test_data = pd.read_csv('/code/mnist/test.csv')

X = train_data.drop('label', axis=1)
Y = train_data['label']

2023-10-08 10:54:17.864066: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-08 10:54:17.909286: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-08 10:54:17.909322: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-08 10:54:17.909356: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-08 10:54:17.917646: I tensorflow/core/platform/cpu_feature_g

In [3]:
X = X.astype('float32') / 255
test_data = test_data.astype('float32') / 255

Y = tf.keras.utils.to_categorical(Y, 10)

In [4]:
train_x = X[:round(X.shape[0] * 0.8)]
check_x = X[round(X.shape[0] * 0.8):]

train_y = Y[:round(Y.shape[0] * 0.8)]
check_y = Y[round(Y.shape[0] * 0.8):]

In [5]:
# from tensorflow.keras.layers import Dense, Dropout

# inputs = tf.keras.Input(shape=(784,))
# x_1 = Dense(256, activation='relu')(inputs)
# x_2 = Dropout(0.2)(x_1)
# outputs = Dense(10, activation='softmax')(x_2)

# model = tf.keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')

In [6]:
train_x = train_x.values.reshape(-1, 28, 28, 1)
check_x = check_x.values.reshape(-1, 28, 28, 1)

test_data = test_data.values.reshape(-1, 28, 28, 1)

In [7]:
train_x.shape

(33600, 28, 28, 1)

In [8]:
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten


inputs = tf.keras.Input(shape=(28, 28, 1))
x_1 = Conv2D(32, kernel_size=(5, 5), strides=1, activation='relu', padding='same')(inputs)
x_2 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(x_1)
x_3 = Flatten()(x_2)
x_4 = Dropout(0.4)(x_3)
outputs = Dense(10, activation='softmax')(x_4)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')

2023-10-08 10:54:24.862065: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-08 10:54:24.891879: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-08 10:54:24.891924: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-08 10:54:24.894160: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-08 10:54:24.894212: I tensorflow/compile

In [9]:
LEARNING_RATE = 0.001
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=[tf.keras.metrics.categorical_accuracy],
)

In [10]:
train_y.shape

(33600, 10)

In [15]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        patience=2, 
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        '/code/mnist/data/temp/mnist_sequential_{epoch:03d}_{val_loss:.4f}.h5',
        save_best_only=True
    ),
    tf.keras.callbacks.TensorBoard(
        log_dir='/code/mnist/logs',
        histogram_freq=1
    ),
]

BATCH_SIZE = 128
EPOCHS = 20
VALIDATION_SPLIT = 0.2

history = model.fit(train_x, train_y, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=VALIDATION_SPLIT, callbacks=callbacks)
model.save('mnist_model.h5')

Epoch 1/20
210/210 [==============================] - 2s 11ms/step - loss: 0.0390 - categorical_accuracy: 0.9881 - val_loss: 0.0604 - val_categorical_accuracy: 0.9807
Epoch 2/20
 15/210 [=>............................] - ETA: 1s - loss: 0.0341 - categorical_accuracy: 0.9885

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


210/210 [==============================] - 2s 10ms/step - loss: 0.0347 - categorical_accuracy: 0.9890 - val_loss: 0.0557 - val_categorical_accuracy: 0.9836
Epoch 3/20
210/210 [==============================] - 2s 11ms/step - loss: 0.0345 - categorical_accuracy: 0.9891 - val_loss: 0.0591 - val_categorical_accuracy: 0.9812
Epoch 4/20
210/210 [==============================] - 2s 10ms/step - loss: 0.0323 - categorical_accuracy: 0.9898 - val_loss: 0.0544 - val_categorical_accuracy: 0.9830
Epoch 5/20
210/210 [==============================] - 2s 10ms/step - loss: 0.0296 - categorical_accuracy: 0.9904 - val_loss: 0.0565 - val_categorical_accuracy: 0.9839
Epoch 6/20
210/210 [==============================] - 2s 10ms/step - loss: 0.0284 - categorical_accuracy: 0.9902 - val_loss: 0.0544 - val_categorical_accuracy: 0.9848
Epoch 7/20
210/210 [==============================] - 2s 10ms/step - loss: 0.0279 - categorical_accuracy: 0.9912 - val_loss: 0.0539 - val_categorical_accuracy: 0.9844
Epoch 8/2

In [12]:
model.summary()

Model: "mnist_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 32)        832       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dropout (Dropout)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 10)                62730     
                                                       

In [13]:
model.evaluate(check_x, check_y)

263/263 [==============================] - 2s 6ms/step - loss: 0.0535 - categorical_accuracy: 0.9833


[0.05348667874932289, 0.9833333492279053]

- 0.9811905026435852
- 0.98416668176651 maxpooling
- 0.9783333539962769 karnel(5,5) => (3,3) & padding 'same'
- 0.9825000166893005 karnel => (3,3)
- 0.9840475916862488 drop 0.2 => 0.4

In [14]:
submit = pd.DataFrame()
submit['ImageId'] = [i for i in range (1,test_data.shape[0]+1)]
submit['Label'] = model.predict(test_data).argmax(axis=1)

submit.to_csv('/code/mnist/submit.csv',index=False)

875/875 [==============================] - 2s 2ms/step
